# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [77]:
import pandas as pd
import numpy as np
from scipy import stats

In [78]:
data = pd.io.stata.read_stata('C:/Users/acer.DESKTOP-NM2UJ22/Springboard/Springboard/EDA_racial_discrimination/data/us_job_market_discrimination.dta')

In [79]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## 1. What test is appropriate for this problem? Does CLT apply?
The call and race are binary variables. 4870 resumes, half with "black sounding names" and half with "white"
If we want to calculate the diff in percentage/proportion of call back for each race, that distibution will be normal (CLT).
We should use a two sample z-test, since we have the entire sample which is larger than 30.
## 2. What are the null and alternate hypotheses?
Null Hypothesis: proportion of resumes with white and black sounding names are the same.
Alt. Hypothesis: proportion ... not the same

In [80]:
tot_b_called = sum(data[data.race=='b'].call)
tot_w_called = sum(data[data.race=='w'].call)
tot_b = len(data[data.race=='b'].race)
p_b = tot_b_called/tot_b
tot_w = len(data[data.race=='w'].race)
p_w = tot_w_called/tot_w
WnB = data.race.size
diff_WnB = p_w-p_b
diff_WnB

0.032032854209445585

## 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [81]:
# Frequentist
# Standard Error
se = np.sqrt(abs((p_b*(1-p_b)/tot_b)+(p_w*(1-p_w)/tot_w)))
print('SE: {:1.4f}'.format(se))
# 95% CI
ci = [p_w-p_b-1.96*se, p_w-p_b+1.96*se]
z = (p_w-p_b)/se
p_val = stats.norm.cdf((1-z))
print('CI: '+str(ci)+', p-value: '+'{:1.10f}'.format(p_val))

SE: 0.0078
CI: [0.016777447859559147, 0.047288260559332024], p-value: 0.0009180097


In [107]:
# Bootstrap
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))
def draw_bs_reps(data, func, size=1):
    bs_replicates = np.empty(size)
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)
    return bs_replicates

# Standard Error
tot_w_call_bs = draw_bs_reps(data[data.race=='w'].call, sum, 10000)
tot_b_call_bs = draw_bs_reps(data[data.race=='b'].call, sum, 10000)
p_w_bs = tot_w_call_bs/tot_w
p_b_bs = tot_b_call_bs/tot_b
tran_w = p_w_bs - p_w + diff_WnB
tran_b = p_b_bs - p_b + diff_WnB
bs_w = sum(tran_w)/len(tran_w)
bs_b = sum(tran_b)/len(tran_b)
se_bs = np.sqrt(abs((bs_w*(1-bs_w)/tot_w)+(bs_b*(1-bs_b)/tot_b)))
print('SE: {:1.4f}'.format(se_bs))
# 95% CI
ci_bs95 = np.percentile(sorted(p_w_bs-p_b_bs),[2.5,97.5])
ci_bs = [ci_bs95[0], ci_bs95[1]]
p_val_bs = np.sum(tran_w-tran_b >= diff_WnB) / 10000
print('CI: '+str(ci_bs)+', p-value: '+'{:1.10f}'.format(p_val_bs))

SE: 0.0050
CI: [0.016837782340862417, 0.047227926078028754], p-value: 0.0000000000


## 4. Write a story describing the statistical significance in the context or the original problem.
Given the specific parameters of our test, where the portions of call backs for black and white sounding names were the only two metrics we compared, the difference in percentage seemed to be significant.
But upon further examination, using the bootstrap method, we see that in fact the outcome of the original dataset was likely randomly skewed, and that by repeating the randomness, we are able to come up with a more accurate range of outcomes.
In conclusion, the proportions are likely the more similar than not.

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
This outcome does not necessarily mean that race/name is or is not a factor in callback success.
This is because there are other variables that also affect the outcome of the callback.
A logistic regression on each variable is needed to examine which specific variables correlate to callback success.